In [1]:
# default_exp core

# arg_navi_tools

> A toolset for .

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#|export

import numpy as np
import scipy as sp
import cv2
from cv2 import aruco
import apriltag
import time
import yaml

import pytransform3d.rotations as pr
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl

import os
import sys
import gdown
from zipfile import ZipFile

from scipy.spatial.transform import Rotation as R
from numpy.linalg import inv

In [4]:
def gdown_unzip(id, filename):
    dataset_url = 'https://drive.google.com/u/1/uc?id=' + id
    dataset_name = filename
        
    if not os.path.isdir(dataset_name):
        gdown.download(dataset_url, output=dataset_name + '.zip', quiet=False)
        zip_file = ZipFile(dataset_name + '.zip')
        zip_file.extractall()
        #zip_file.extractall(dataset_name) # depends on how to zip it
        zip_file.close()

In [5]:
id = '1DN5tiV_WII_aJM3DLijPnq5OLtI0HAq1'
filename = "ViperX_apriltags"
gdown_unzip(id, filename)

Downloading...
From: https://drive.google.com/u/1/uc?id=1DN5tiV_WII_aJM3DLijPnq5OLtI0HAq1
To: /home/arg/arg_navi_tools/ViperX_apriltags.zip
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48.1M/48.1M [00:23<00:00, 2.03MB/s]


In [8]:
class camera_projection:
    def __init__(self):
        self.id = '1DN5tiV_WII_aJM3DLijPnq5OLtI0HAq1'
        self.filename = "ViperX_apriltags"
        self.camera_info_path = 'ViperX_apriltags/camera_info.yaml'
        self.img_path = 'ViperX_apriltags/rgb/'
        self.depth_path = 'ViperX_apriltags/depth/'
        self.identity = np.identity(4)
        self.tag_size = 0.0415
        self.s = 0.5 * self.tag_size
        self.joints = ['Joint_1_pose', 'Joint_2_pose', 
                       'Joint_3_pose', 'Joint_4_pose', 
                       'Joint_5_pose', 'Joint_6_pose']
        
    def read_camera_info(self):
        with open(self.camera_info_path, "r") as stream:
            try:
                camera_data = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        self.camera_matrix = np.array(camera_data['camera_matrix']['data'])
        self.camera_matrix = self.camera_matrix.reshape(3, 3)
        self.dist_coeffs = np.array(camera_data['distortion_coefficients']['data'])
        self.dist_coeffs = self.dist_coeffs.reshape(1, 5)
        self.cameraParams_Intrinsic = [self.camera_matrix[0,0], self.camera_matrix[1,1], 
                                       self.camera_matrix[0,2], self.camera_matrix[1,2]]
        # cameraParams_Intrinsic = [camera_fx, camera_fy, 
        #                           camera_cx, camera_cy]
        
    def read_images(self, idx):
        self.img_path = self.img_path + str(idx)
        self.depth_path = self.depth_path + str(idx)
        self.img = cv2.imread(self.img_path)
        self.gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        self.depth = cv2.imread(self.depth_path, -cv2.IMREAD_ANYDEPTH)
        self.img_dst = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    def apriltag_detection(self):
        print("[INFO] detecting AprilTags...")
        options = apriltag.DetectorOptions(families="tag36h11")
        detector = apriltag.Detector(options)
        #results = detector.detect(gray)
        self.detection_results, dimg = detector.detect(self.gray, return_image=True)
        print("[INFO] {} total AprilTags detected".format(len(self.detection_results)))
        
    def solvePnP(self):
        img_pts = self.detection_results[0].corners.reshape(1,4,2)
        obj_pt1 = [-s, -s, 0.0]
        obj_pt2 = [ s, -s, 0.0]
        obj_pt3 = [ s,  s, 0.0]
        obj_pt4 = [-s,  s, 0.0]
        obj_pts = obj_pt1 + obj_pt2 + obj_pt3 + obj_pt4
        obj_pts = np.array(obj_pts).reshape(4,3)
        # print(img_pts)
        # print(obj_pts)
        _, r_vec, t_vec = cv2.solvePnP(obj_pts, img_pts, self.camera_matrix, 
                                       self.dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
        R_mat, _ = cv2.Rodrigues(r_vec)
        T = np.hstack((R_mat, t_vec)).reshape(3,4)
        tag_pose = np.vstack((T, [0,0,0,1])).reshape(4,4)
        dist = np.linalg.norm(t_vec)